[source](https://www.learningeeg.com/montages-and-technical-components)

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pywt
import librosa

# Define EEG features and names
NAMES = ['LL', 'LP', 'RP', 'RR']
FEATS = [['Fp1', 'F7', 'T3', 'T5', 'O1'],
         ['Fp1', 'F3', 'C3', 'P3', 'O1'],
         ['Fp2', 'F8', 'T4', 'T6', 'O2'],
         ['Fp2', 'F4', 'C4', 'P4', 'O2']]

# Define directory for EEG spectrograms
directory_path = 'EEG_Spectrograms/'
os.makedirs(directory_path, exist_ok=True)

# Function to denoise EEG signals
def denoise(x, wavelet='haar', level=1):
    coeff = pywt.wavedec(x, wavelet, mode="per")
    sigma = (1 / 0.6745) * np.mean(np.abs(coeff[-level] - np.mean(coeff[-level])))
    uthresh = sigma * np.sqrt(2 * np.log(len(x)))
    coeff[1:] = (pywt.threshold(i, value=uthresh, mode='hard') for i in coeff[1:])
    return pywt.waverec(coeff, wavelet, mode='per')

# Function to generate spectrogram from EEG data
def spectrogram_from_eeg(parquet_path, display=False):
    eeg = pd.read_parquet(parquet_path)
    middle = (len(eeg) - 10_000) // 2
    eeg = eeg.iloc[middle:middle + 10_000]
    img = np.zeros((128, 256, 4), dtype='float32')

    signals = []
    for k, cols in enumerate(FEATS):
        for kk in range(4):
            x = eeg[cols[kk]].values - eeg[cols[kk + 1]].values
            x = np.nan_to_num(x, nan=np.nanmean(x)) if not np.isnan(x).all() else np.zeros_like(x)
            signals.append(x)
            if USE_WAVELET:
                x = denoise(x, wavelet=USE_WAVELET)
            mel_spec = librosa.feature.melspectrogram(y=x, sr=200, hop_length=len(x) // 256,
                                                      n_fft=1024, n_mels=128, fmin=0, fmax=20, win_length=128)
            mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max).astype(np.float32)[:,:256]
            mel_spec_db = (mel_spec_db + 40) / 40
            img[:,:,k] += mel_spec_db

        img[:,:,k] /= 4.0

    if display:
        plt.figure(figsize=(12, 6))
        plt.subplot(1, 2, 1)
        plt.imshow(np.concatenate([img[:,:,i] for i in range(4)], axis=1), aspect='auto', origin='lower')
        plt.title('Combined Spectrograms')

        plt.subplot(1, 2, 2)
        for k in range(4):
            offset = k * 1000
            plt.plot(range(10000), signals[k] + offset, label=NAMES[k])
        plt.legend()
        plt.title('Combined Signals')
        plt.tight_layout()
        plt.show()

    return img

# Main execution
PATH = '/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/'
DISPLAY = 4
EEG_IDS = train.eeg_id.unique()
all_eegs = {}

for i, eeg_id in enumerate(EEG_IDS):
    if (i % 100 == 0) and (i != 0):
        print(i, ', ', end='')

    # Create spectrogram from EEG parquet
    img = spectrogram_from_eeg(f'{PATH}{eeg_id}.parquet', i < DISPLAY)

    # Save to disk
    if i == DISPLAY:
        print(f'Creating and writing {len(EEG_IDS)} spectrograms to disk... ', end='')
    np.save(f'{directory_path}{eeg_id}', img)
    all_eegs[eeg_id] = img

np.save('eeg_specs', all_eegs)